# Rest Service Demonstration

## DB Sections Request

In [ ]:
from dask import dataframe as dd
from datetime import datetime
import pandas as pd

df = dd.read_parquet('../cache/solarpanel.parquet')
start_time = pd.to_datetime(datetime.fromtimestamp(1642686469405 / 1000))
end_time = pd.to_datetime(datetime.fromtimestamp(1646364869405 / 1000))

mask = (
  (df.index >= start_time) &
  (df.index <= end_time)
)

mf = df.loc[mask, []]
mf = mf.reset_index()
mf["gap"] = mf["timestamp"].diff().dt.seconds > 5
mf["gap"] = mf["gap"].cumsum()

mf.compute()

NotImplementedError: Partition size is less than overlapping window size. Try using ``df.repartition`` to increase the partition size.

In [132]:
import numpy as np

gk = mf.groupby("gap").agg({'timestamp' : [np.min, np.max]})
gk.compute()

Dask DataFrame Structure:
                    timestamp                
                         amin            amax
npartitions=1                                
               datetime64[ns]  datetime64[ns]
                          ...             ...
Dask Name: aggregate-agg, 4583 tasks

In [131]:
gk.compute().to_numpy().astype('datetime64[s]').astype('int')

array([[1645444759, 1645446079],
       [1645446088, 1645447661],
       [1645447851, 1645449299]])

## DB Records Request

In [11]:
from dask import dataframe as dd
from datetime import datetime
import pandas as pd

df = dd.read_parquet('../cache/solarpanel.parquet')
start_time = pd.to_datetime(datetime.fromtimestamp(1646906864141 / 1000))
end_time = pd.to_datetime(datetime.fromtimestamp(1646908343348 / 1000))
print(start_time)
print(end_time)

mask = (
  (df.index >= start_time) &
  (df.index <= end_time)
)

units = ["voltage", "input_current"]

mf = df.loc[mask, units]

if len(mf.index) > 100:
    resample_freq = len(mf) // 100
    print(len(mf))
    mf = mf.resample(str(resample_freq) + "S").mean()

    mf = mf.dropna()

mf["timestamp"] = mf.index
mf["gap"] = mf["timestamp"].diff().dt.seconds > resample_freq
mf["gap"] = mf["gap"].cumsum()
mf.compute()

2022-03-10 11:07:44.141000
2022-03-10 11:32:23.348000
1479


NotImplementedError: Partition size is less than overlapping window size. Try using ``df.repartition`` to increase the partition size.

In [7]:
gk = mf.groupby("gap")["timestamp"].count().cumsum()
split_rows = gk.compute().tolist()
split_rows

[21, 126]

In [8]:
import numpy as np

mf = mf[units].reset_index().compute()
mf["timestamp"] = mf["timestamp"].values.astype("datetime64[s]").astype("int")
mf[units] = mf[units].applymap('{:,.2f}'.format)

dfs = np.split(mf, split_rows, axis=0)
dfs

[     timestamp voltage input_current
 0   1646912595   12.90         26.26
 1   1646912596   13.13         20.78
 2   1646912597   13.02         30.59
 3   1646912598   13.59         84.10
 4   1646912599   13.57         22.17
 5   1646912600   13.84        132.36
 6   1646912601   13.65          2.29
 7   1646912602   13.32         58.81
 8   1646912603   13.54         14.09
 9   1646912604   13.70         94.59
 10  1646912605   13.64         76.57
 11  1646912606   13.40        108.32
 12  1646912607   12.87        130.31
 13  1646912608   14.17         44.19
 14  1646912609   13.21         42.45
 15  1646912610   13.04         54.04
 16  1646912611   12.82         14.41
 17  1646912612   13.68         24.38
 18  1646912613   13.30         77.85
 19  1646912614   13.86         40.85
 20  1646912615   12.92         44.86,
      timestamp voltage input_current
 0   1646912636   13.57         84.81
 1   1646912637   13.94        115.31
 2   1646912638   14.25         22.63
 3   164691

In [9]:
mfs = []

for i in range(len(dfs) - 1):
      mfs.append(dfs[i].to_numpy().tolist())
        
mfs

[[[1646912595, '12.90', '26.26'],
  [1646912596, '13.13', '20.78'],
  [1646912597, '13.02', '30.59'],
  [1646912598, '13.59', '84.10'],
  [1646912599, '13.57', '22.17'],
  [1646912600, '13.84', '132.36'],
  [1646912601, '13.65', '2.29'],
  [1646912602, '13.32', '58.81'],
  [1646912603, '13.54', '14.09'],
  [1646912604, '13.70', '94.59'],
  [1646912605, '13.64', '76.57'],
  [1646912606, '13.40', '108.32'],
  [1646912607, '12.87', '130.31'],
  [1646912608, '14.17', '44.19'],
  [1646912609, '13.21', '42.45'],
  [1646912610, '13.04', '54.04'],
  [1646912611, '12.82', '14.41'],
  [1646912612, '13.68', '24.38'],
  [1646912613, '13.30', '77.85'],
  [1646912614, '13.86', '40.85'],
  [1646912615, '12.92', '44.86']],
 [[1646912636, '13.57', '84.81'],
  [1646912637, '13.94', '115.31'],
  [1646912638, '14.25', '22.63'],
  [1646912639, '12.89', '146.27'],
  [1646912640, '12.65', '86.26'],
  [1646912641, '13.59', '126.34'],
  [1646912642, '13.56', '121.65'],
  [1646912643, '12.85', '56.35'],
  [1646